# Inputs
---

In [1]:
import numpy as np, zarr, nrrd
# alignment functions
from bigstream.align import alignment_pipeline
from bigstream.transform import apply_transform
import tifffile
import seaborn as sns

In [2]:
p      = '/u/home/f/f7xiesnm/project-zipursky/easifish/lt185/outputs/'
outdir = '/u/home/f/f7xiesnm/project-zipursky/easifish/results/test_bigstream_lt185_r1tor2'
!mkdir -p $outdir

In [3]:
fix_path = p + '/r2v3/stitching/export.n5'
mov_path = p + '/r1v3/stitching/export.n5'
exp_factor = 2  # replace this with the known expansion factor for your sample to use pre-expansion units

fix_spacing = np.array([0.42, 0.23, 0.23]) / exp_factor
mov_spacing = np.array([0.42, 0.23, 0.23]) / exp_factor
s1_scale = [1,2,2] # z,y,x
s3_scale = [4,8,8]
s4_scale = [8,16,16]

# load fix data and spacing
fix_zarr = zarr.open(store=zarr.N5Store(fix_path), mode='r')
# fix_meta = fix_zarr.attrs.asdict()
# fix_spacing = np.array(fix_meta['pixelResolution']['dimensions'][::-1]) / exp_factor
fix_spacing_s1 = fix_spacing * s1_scale 
fix_spacing_s3 = fix_spacing * s3_scale 
fix_spacing_s4 = fix_spacing * s4_scale 

# load mov data and spacing
mov_zarr = zarr.open(store=zarr.N5Store(mov_path), mode='r')
# mov_meta = mov_zarr.attrs.asdict()
# mov_spacing = np.array(mov_meta['pixelResolution']['dimensions'][::-1]) / exp_factor
mov_spacing_s1 = mov_spacing * s1_scale
mov_spacing_s3 = mov_spacing * s3_scale 
mov_spacing_s4 = mov_spacing * s4_scale 

# print spacings
print(fix_spacing, mov_spacing)
print(fix_spacing_s1, mov_spacing_s1)
print(fix_spacing_s3, mov_spacing_s3)
print(fix_spacing_s4, mov_spacing_s4)

# print shapes
print(fix_zarr['/c3_reg/s3'].shape, mov_zarr['/c3_reg/s3'].shape)
print(fix_zarr['/c3_reg/s4'].shape, mov_zarr['/c3_reg/s4'].shape)


[0.21  0.115 0.115] [0.21  0.115 0.115]
[0.21 0.23 0.23] [0.21 0.23 0.23]
[0.84 0.92 0.92] [0.84 0.92 0.92]
[1.68 1.84 1.84] [1.68 1.84 1.84]
(706, 902, 1128) (756, 901, 1130)
(353, 451, 564) (378, 450, 565)


# Alignment
---

### global

In [4]:
# alignment functions
from bigstream.align import alignment_pipeline
from bigstream.transform import apply_transform

In [5]:
# get global alignment channels
fix = fix_zarr['/c3_reg/s4'][...]
mov = mov_zarr['/c3_reg/s4'][...][:,::-1,::-1]

fix[:45] = 0
fix[-40:] = 0

mov[:65] = 0
mov[-40:] = 0

# # create some masks to focus alignment only on planes below the large artifact at the top
# fix_mask = np.ones(fix.shape, dtype=np.uint8)
# fix_mask[:45] = 0
# fix_mask[-40:] = 0

# mov_mask = np.ones(mov.shape, dtype=np.uint8)
# mov_mask[:65] = 0
# mov_mask[-40:] = 0

In [6]:
tifffile.imwrite(outdir+'/fix.tiff', fix)
tifffile.imwrite(outdir+'/mov.tiff', mov)

In [7]:

%%time

# define alignment steps
ransac_kwargs = {
    'blob_sizes': [2, 8],
    'cc_radius': 12,
    'match_threshold':0.4,
    'nspots':10000,
    # 'nspots':2000,
}

affine_kwargs = {
    'alignment_spacing':4.0,
    'shrink_factors':(1,),
    'smooth_sigmas':(4.,),
    'optimizer_args':{
        'learningRate':0.25,
        'minStep':0.,
        'numberOfIterations':400,
    },
    'metric':'C',
}


steps = [
    ('ransac', ransac_kwargs,),
    ('affine', affine_kwargs,),
        ]

# align
affine = alignment_pipeline(
    fix, mov,
    fix_spacing_s4,
    mov_spacing_s4,
    steps,
    # fix_mask=fix_mask,
    # mov_mask=mov_mask,
)


computing fixed spots
found 109604 fixed spots
computing moving spots
found 118574 moving spots
sorting spots
extracting contexts
computing pairwise correlations
645 matched spots
aligning
LEVEL:  0  ITERATION:  0  METRIC:  -0.11302767720429235
LEVEL:  0  ITERATION:  1  METRIC:  -0.11370625776491895
LEVEL:  0  ITERATION:  2  METRIC:  -0.11421068087425387
LEVEL:  0  ITERATION:  3  METRIC:  -0.11455643560358587
LEVEL:  0  ITERATION:  4  METRIC:  -0.11482885115226282
LEVEL:  0  ITERATION:  5  METRIC:  -0.11520105190361345
LEVEL:  0  ITERATION:  6  METRIC:  -0.11551555716003026
LEVEL:  0  ITERATION:  7  METRIC:  -0.11581631945043996
LEVEL:  0  ITERATION:  8  METRIC:  -0.11611058071768068
LEVEL:  0  ITERATION:  9  METRIC:  -0.11640045589741846
LEVEL:  0  ITERATION:  10  METRIC:  -0.11668716330759266
LEVEL:  0  ITERATION:  11  METRIC:  -0.11697176478267193
LEVEL:  0  ITERATION:  12  METRIC:  -0.1172551352617103
LEVEL:  0  ITERATION:  13  METRIC:  -0.11753664016048211
LEVEL:  0  ITERATION:  1

In [8]:
print(affine)

# apply affine only
affine_aligned = apply_transform(
    fix, mov,
    fix_spacing_s4, mov_spacing_s4,
    transform_list=[affine,],
)

# write results
np.savetxt(outdir+'/affine-masked.mat', affine)
tifffile.imwrite(outdir+'/affine-masked.tiff', affine_aligned)

[[ 1.00413431e+00 -6.81085054e-02  1.33921626e-01  2.34078213e+00]
 [ 6.76878465e-02  9.63119858e-01  1.10307962e-01 -6.61707970e+01]
 [-1.70472456e-01 -1.26244223e-02  9.96912545e-01  1.37280452e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
